In [1]:
#conda install gensim or pip install gensim
import pandas as pd

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from nltk.corpus import stopwords
import itertools
import functools
import gensim
from functools import reduce
nltk.download('gutenberg')
from nltk.corpus import gutenberg
from nltk.sentiment import SentimentAnalyzer
nltk.download('vader_lexicon')
import numpy as np
from scipy import spatial
import re
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras

C:\Users\aayus\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\aayus\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aayus\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Using TensorFlow backend.


In [2]:
filepath = "./emailData/emails.csv"
# Read the data into a pandas dataframe called emails
emails = pd.read_csv(filepath)
headers = [header for header in emails.columns]
print("Successfully loaded {} rows and {} columns!".format(emails.shape[0], emails.shape[1]))
print(display(emails.head()))

Successfully loaded 517401 rows and 2 columns!


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


None


In [4]:
print(emails.loc[0]["message"])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [5]:
import email

def insert_value(dictionary, key, value):
    if key in dictionary:
        values = dictionary.get(key)
        values.append(value)
        dictionary[key] = values
    else:
        dictionary[key] = [value]
    return dictionary

def get_headers(df, header_names):
    headers = {}
    messages = df["message"]
    for message in messages:
        e = email.message_from_string(message)
        for item in header_names:
            header = e.get(item)
            insert_value(dictionary = headers, key = item, value = header) 
    print("Successfully retrieved header information!")
    return headers
header_names = ["Date", "Subject", "X-Folder", "X-From", "X-To"]    
headers = get_headers(emails, header_names)

Successfully retrieved header information!


In [6]:
def get_messages(df):
    messages = []
    for item in df["message"]:
        # Return a message object structure from a string
        e = email.message_from_string(item)    
        # get message body  
        message_body = e.get_payload()
        message_body = message_body.lower()
        messages.append(message_body)
    print("Successfully retrieved message body from e-mails!")
    return messages
msg_body = get_messages(emails)
emails["Message-Body"] = msg_body

Successfully retrieved message body from e-mails!


In [7]:
emails["Message-Body"].head(10)

0                            here is our forecast\n\n 
1    traveling to have a business meeting takes the...
2                       test successful.  way to go!!!
3    randy,\n\n can you send me a schedule of the s...
4                  let's shoot for tuesday at 11:45.  
5    greg,\n\n how about either next tuesday or thu...
6    please cc the following distribution list with...
7                     any morning between 10 and 11:30
8    1. login:  pallen pw: ke9davis\n\n i don't thi...
9    ---------------------- forwarded by phillip k ...
Name: Message-Body, dtype: object

In [8]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

C:\Users\aayus\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
import tensorflow as tf

In [10]:
vocab = set()

In [11]:
def processText(text):
    
    sentences = nltk.sent_tokenize(text)
    tokens = []
        
    for sentence in sentences:
        
        #Remove URL
        sentence = re.sub(r"http\S+", "", sentence)
        
        #words = nltk.word_tokenize(sentence)
        words = tknzr.tokenize(sentence)
        #Converting to LowerCase
        words = map(str.lower, words)
    
        words = map(lambda x: re.sub(r'[^a-zA-Z]', "", x), words)
        
        #punctuations = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~.?!'
        #words = map(lambda x: x.translate(str.maketrans('', '', punctuations)), words)
        
        # Remove empty strings
        words = filter(lambda x: len(x) > 0, words)
        
        #Remove words which are not part of word2vec vocab
        words = filter(lambda x: x in word2vec.vocab, words)
        
        words = list(words)
        
        #Add the words to the vocabulary
        vocab.update(words)
        
        # Adding the preprocessed words to the document
        tokens = tokens + words
        
    return tokens

In [13]:
emails['message_tokens'] = emails['Message-Body'].apply(lambda x: processText(x))

In [14]:
emails['message_tokens'].head()

0                            [here, is, our, forecast]
1    [traveling, have, business, meeting, takes, th...
2                          [test, successful, way, go]
3    [randy, can, you, send, me, schedule, the, sal...
4                      [lets, shoot, for, tuesday, at]
Name: message_tokens, dtype: object

In [18]:
#Save the emailTokens file
emails['message_tokens'].to_pickle('./emailTokens')

In [19]:
#Save the vocab file
with open('vocab.txt','w') as f:
    f.write(str(vocab))